In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Kidney_Disease_Classification_DL'

In [4]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/G-Sahil123/Kidney_Disease_Classification_DL.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "G-Sahil123"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "ed797bbf811f5c2714a6f8ee4b3cc86b7d597214"

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json


In [10]:

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=Path("artifacts/training/model.h5"),
            training_data=Path("artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone"),
            mlflow_uri="https://dagshub.com/G-Sahil123/Kidney_Disease_Classification_DL.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [ ]:
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from mlflow.tracking import MlflowClient

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

            from mlflow.tracking import MlflowClient

    def stage_model(self, model_name: str = "VGG16Model", stage: str = "Staging"):
        client = MlflowClient()

        # Get the latest version of the registered model
        latest_version_info = client.get_latest_versions(model_name, stages=["None"])
        if latest_version_info:
            version = latest_version_info[0].version

            # Transition to desired stage
            client.transition_model_version_stage(
                name=model_name,
                version=version,
                stage=stage,
                archive_existing_versions=False
            )
            print(f"✅ Model {model_name} version {version} moved to {stage} stage.")
        else:
            print(f"⚠️ No versions of {model_name} found to transition.")    

In [ ]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
    evaluation.stage_model(model_name="VGG16Model",stage="Staging")

except Exception as e:
   raise e

[2025-09-15 21:49:59,245: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-15 21:49:59,245: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-15 21:49:59,262: INFO: common: created directory at: artifacts]
Found 3732 images belonging to 4 classes.
117/117 [==============================] - 461s 4s/step - loss: 1.1672 - accuracy: 0.7460
[2025-09-15 21:57:41,322: INFO: common: json file saved at: scores.json]


2025/09/15 21:57:45 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-09-15 21:57:48,470: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\anssa\AppData\Local\Temp\tmphxq_hy5i\model\data\model\assets
[2025-09-15 21:57:50,610: INFO: builder_impl: Assets written to: C:\Users\anssa\AppData\Local\Temp\tmphxq_hy5i\model\data\model\assets]


c:\Kidney_Disease_Classification_DL\myvenv2\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/09/15 22:01:36 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 3
Created version '3' of model 'VGG16Model'.


In [14]:
from mlflow.tracking import MlflowClient

def stage_model(self, model_name: str = "VGG16Model", stage: str = "Staging"):
    client = MlflowClient()

    # Get the latest version of the registered model
    latest_version_info = client.get_latest_versions(model_name, stages=["None"])
    if latest_version_info:
        version = latest_version_info[0].version

        # Transition to desired stage
        client.transition_model_version_stage(
            name=model_name,
            version=version,
            stage=stage,
            archive_existing_versions=False
        )
        print(f"Model {model_name} version {version} moved to {stage} stage.")
